In [1]:
import sys
import time
import json 
import pandas as pd
from app.builtin import *

sys.path.append('../app')  # for let gsdmm to work

Using TensorFlow backend.


In [2]:
models= {
    'ntm': NtmModel,
    'lda': LdaModel,
    'lftm': LftmModel,
    'gsdmm': GsdmmModel
}

# 20 NG Dataset

In [3]:
dataset = '../data/20ng.txt'
labels = '../data/20ng_labels.txt'

In [4]:
with open(labels, "r") as datafile:
    true_topics = [line.rstrip() for line in datafile if line]
true_topics[0:10]

['rec.autos',
 'comp.sys.mac.hardware',
 'comp.sys.mac.hardware',
 'comp.graphics',
 'sci.space',
 'talk.politics.guns',
 'sci.med',
 'comp.sys.ibm.pc.hardware',
 'comp.os.ms-windows.misc',
 'comp.sys.mac.hardware']

In [31]:
results = {}
for name, model in models.items():
    start = time.time()
    m = model()
    print(name)
    if name == 'gsdmm':
        m.corpus = dataset
    results[name] = m.get_corpus_predictions()
    dur = time.time() - start
    print('Done in ' + str(dur))

ntm


/Users/pasquale/git/ted-talk-topic-extraction/app/builtin/doc2topic/models.py:18: RuntimeWarning: invalid value encountered in true_divide
  L1normalize = (lambda x: x/L1(x))


Done in 14.28440809249878
lda
Done in 4.146892070770264
lftm
Done in 3.477168083190918
gsdmm
Done in 24.56701111793518


In [33]:
df = pd.DataFrame()
df['true'] = true_topics
for name, model in models.items():
    r = results[name]
    for i in range(0,3):
        key, value = zip(*[x[i] if len(x) > i else (-1, -1) for x in r])
        df['%s_topic_%d'%(name,i)] = key
        df['%s_score_%d'%(name,i)] = value
df

ValueError: Length of values does not match length of index

In [7]:
df.index.name = 'id'
df.to_csv('./out/predictions_20ng.csv')

In [8]:
for name, model in models.items():
    m = model()
    with open('./out/'+name+'_topics_20ng.json', 'w') as file:
        json.dump(m.topics(), file)

# AFP dataset

In [3]:
dataset = '../data/afp.txt'
labels = '../data/afp_labels.txt'

In [4]:
with open(labels, "r") as datafile:
    true_topics = [line.rstrip() for line in datafile if line]
true_topics[0:10]

[',,',
 '13000000,13007000 13006000 13008000,',
 '11000000,11014000 11001000 11002000 11006000,',
 '11000000,11014000 11001000 11002000 11006000,',
 '11000000,11014000 11001000 11002000 11006000,',
 '11000000,11014000 11001000 11002000 11006000,',
 '15000000,,',
 '11000000,11014000 11001000 11002000 11006000,',
 '02000000,02003000 02001000 16001000,02003001',
 '15000000,15054000,']

In [5]:
true1, true2, true3 = zip(*[line.split(',') for line in true_topics])
true1[0:10]

('',
 '13000000',
 '11000000',
 '11000000',
 '11000000',
 '11000000',
 '15000000',
 '11000000',
 '02000000',
 '15000000')

In [6]:
results = {}
for name, model in models.items():
    start = time.time()
    m = model()
    print(name)
    if name == 'gsdmm':
        m.corpus = dataset
    results[name] = m.get_corpus_predictions()
    dur = time.time() - start
    print('Done in ' + str(dur))

W0410 11:16:51.449277 4644371904 deprecation.py:506] From /usr/local/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


ntm


W0410 11:16:51.633363 4644371904 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow_core/python/ops/nn_impl.py:183: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0410 11:16:52.052992 4644371904 module_wrapper.py:139] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

/Users/pasquale/git/ted-talk-topic-extraction/app/builtin/doc2topic/models.py:18: RuntimeWarning: invalid value encountered in true_divide
  L1normalize = (lambda x: x/L1(x))


Done in 5.643973112106323
lda
Done in 3.8390939235687256
lftm
Done in 3.3734350204467773
gsdmm
Done in 263.6128351688385


In [7]:
df = pd.DataFrame()
df['true'] = true1
df['true2'] = true2
df['true3'] = true3
for name, model in models.items():
    r = results[name]
    for i in range(0,3):
        key, value = zip(*[x[i] if len(x) > i else (-1, -1) for x in r])
        df['%s_topic_%d'%(name,i)] = key
        df['%s_score_%d'%(name,i)] = value
df

,true,true2,true3,ntm_topic_0,ntm_score_0,ntm_topic_1,ntm_score_1,ntm_topic_2,ntm_score_2,lda_topic_0,...,lftm_topic_1,lftm_score_1,lftm_topic_2,lftm_score_2,gsdmm_topic_0,gsdmm_score_0,gsdmm_topic_1,gsdmm_score_1,gsdmm_topic_2,gsdmm_score_2
0,,,,9,0.342370,1,0.323482,16,0.244672,0,...,1,0.006803,2,0.006803,16,1.000000,10,1.872155e-10,8,5.106525e-14
1,13000000,13007000 13006000 13008000,,5,0.296621,9,0.129710,1,0.121188,0,...,7,0.251570,3,0.055781,16,1.000000,10,4.297185e-18,8,1.060044e-19
2,11000000,11014000 11001000 11002000 11006000,,13,0.694453,7,0.292373,0,0.007895,0,...,15,0.179487,2,0.094017,16,1.000000,10,1.964922e-09,8,6.808742e-13
3,11000000,11014000 11001000 11002000 11006000,,13,0.420780,7,0.325828,8,0.231620,0,...,3,0.196262,15,0.196262,16,0.987579,10,1.135475e-02,8,1.066308e-03
4,11000000,11014000 11001000 11002000 11006000,,7,0.413961,13,0.412301,16,0.135707,0,...,15,0.165354,10,0.086614,16,1.000000,10,6.518211e-09,8,1.216812e-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125511,16000000,16002000 16001000,,9,0.410465,2,0.200190,11,0.109299,0,...,12,0.093545,3,0.055575,16,1.000000,10,2.342492e-13,8,5.643892e-17
125512,01000000 08000000,01016000 08006000 01005000 01007000 01026000,08003002 08003003,10,0.280207,9,0.143741,11,0.138465,0,...,8,0.048621,3,0.045778,16,0.999997,10,2.684987e-06,8,2.386810e-12
125513,01000000 02000000,01026000,,12,0.228517,8,0.194645,2,0.189475,0,...,5,0.071784,9,0.036247,16,1.000000,10,5.209413e-13,8,1.791036e-17
125514,14000000 08000000 11000000,14023000 08003000 12006000,,13,0.275202,3,0.211507,12,0.205389,0,...,12,0.250592,10,0.169428,16,1.000000,10,8.215579e-08,8,2.295016e-09


In [10]:
df.index.name = 'id'
df.to_csv('./out/predictions_afp.csv')

In [11]:
for name, model in models.items():
    m = model()
    with open('./out/'+name+'_topics_afp.json', 'w') as file:
        json.dump(m.topics(), file)